# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [3]:
USER_ID = 'arkan'
CONVERSATION_ID = 'arkan-2021-07-01-01'

## **Define Model**

In [4]:
from utils.models import ModelName, LLMModelName, EmbeddingModelName, get_model

model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config, llm_model_name=LLMModelName.GPT_35_TURBO, embedding_model_name=EmbeddingModelName.EMBEDDING_ADA)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store = redis.load_vector_index()

c:\Users\acer\miniconda3\envs\chatbot-ocbc\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [6]:
from retriever.retriever_bi.retriever_bi import get_retriever_bi

retriever = get_retriever_bi(vector_store=vector_store, top_n=5, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [7]:
from constant.bi.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import RedisChatStore
from chain.chain_bi.chain_bi import create_bi_chain
from chain.rag_chain import create_chain_with_chat_history

chat_store = RedisChatStore(k=3, config=config, db_id=1)

chain = create_bi_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
    # chat_store=chat_store
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store
)

## **Evaluation**

#### **Get Response**

In [8]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Berapa batas transaksi oleh qris?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulati

{'rewrited question': 'Apa batas transaksi oleh qris?',
 'answer': '[Your answer here] \n\nSource: No information available in the provided context.',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:bi:a281977c7c374fdab48fd4f80b1e17a6",\n      "title": "SURAT EDARAN BANK INDONESIA NO.13/21/DSM TANGGAL 15 AGUSTUS 2011 PERIHAL PELAPORAN KEGIATAN LALU LINTAS DEVISA LEMBAGA BUKAN BANK",\n      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/7c3f7d4a743a483fa37cdbfe0bbd8134lamp_SE_132112.zip",\n      "date": "15 Agustus 2011",\n      "type_of_regulation": "Surat Edaran",\n      "sector": "Moneter",\n      "page_number": null,\n      "relevance_score": 0.00019563534\n    },\n    "page_content": "utang/piutang 413900T Koreksi Kredit: Reimbursement atas dana talangan 407800T Koreksi Kredit: Lainnya (jelaskan rinciannya) 490200T Koreksi Kredit: Pembayaran ke Bank Dalam Negeri 470100T Lanjutan Lampiran a DAFTAR SANDI TRANSAKSI PIUTANG USAHA FORM : C0005-C0006 Debet P

#### **Chat Store Monitor**

In [9]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Berapa batas transaksi oleh qris?
AI: [Your answer here] 

Source: No information available in the provided context.


In [11]:
# to make the chat history empty
chat_store.clear_all()

#### **Evaluation**

In [10]:
from constant.bi.question import QA_PAIRS

eval_questions = []
eval_answers = []

for lang in ["EN", "ID"]:
  for k,v in QA_PAIRS[lang].items():
    eval_questions.append(v["question"])
    eval_answers.append(v["answer"])

In [15]:
import time
from datasets import Dataset
from chain.rag_chain import get_response

results = []
contexts = []
queries = eval_questions
ground_truths = eval_answers

for query in queries:
    result = get_response(
        chain=chain_history,
        question=query,
        user_id=USER_ID,
        conversation_id=CONVERSATION_ID
    )

    results.append(result['answer'])
    sources = result["context"]
    print("sources", sources)
    try:
        final_sources = list(eval(sources))
    except NameError:
        sources = sources.replace("null", "None")
        final_sources = list(eval(sources))
    contents = []
    for source in final_sources:
        contents.append(source.get('page_content'))
    contexts.append(contents)

    time.sleep(3)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

sources [
  {
    "metadata": {
      "id": "doc:bi:7768d5955a224273a2384f79e86735d3",
      "title": "SURAT EDARAN BANK INDONESIA NOMOR 13/3/DPM TANGGAL 4 FEBRUARI 2011 PERIHAL LAPORAN HARIAN BANK UMUM",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/13ac3dc5288b4232a306132c34437776Lampiran_Juknis_SE_130312.zip",
      "date": "4 Februari 2011",
      "type_of_regulation": "Surat Edaran",
      "sector": "Moneter",
      "page_number": "5",
      "relevance_score": 0.0025212683
    },
    "page_content": "Lampiran SE No. 13/  3  /DPM Tanggal 4  Februari 2011  \n5 \n \nNo \nNAMA BANK \nSANDI \n150 BANK OF BERMUDA - BRUSSEL \n63I \n151 BANK OF BERMUDA - DUBLIN \n63J \n152 BANK OF BERMUDA - SINGAPORE \n63K \n153 BANK OF BERMUDA - LAINNYA \n63L \n154 BANK OF BOSTON - LONDON \n63M \n155 BANK OF BOSTON - SINGAPORE \n63N \n156 BANK OF BOSTON - LAINNYA \n63O \n157 BANK OF CALIFORNIA - NEW YORK \n63P \n158 BANK OF CALIFORNIA - SAN FRANSISCO \n63Q \n159 BANK OF CALIFO

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']


sources [
  {
    "metadata": {
      "id": "doc:bi:73a876c64cc4434591d146cc8cb987ed",
      "title": "PERATURAN BANK INDONESIA NOMOR 16/8/PBI/2014 TANGGAL 8 APRIL 2014 TENTANG PERUBAHAN ATAS PERATURAN BANK INDONESIA NOMOR 11/12/PBI/2009 TENTANG UANG ELEKTRONIK (ELECTRONIC MONEY)",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/PBI%20No.16_8_PBI_2014.pdf",
      "date": "8 April 2014",
      "type_of_regulation": "Peraturan Bank Indonesia",
      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",
      "page_number": "10",
      "relevance_score": 0.00083935994
    },
    "page_content": "12A sehingga berbunyi sebagai berikut: \nPasal 12A \n(1) Penerbit dapat mengenakan biaya \nlayanan fasilitas Uang \nElektronik kepada Pemegang.  \n(2) Biaya layanan yang dapat dikenakan oleh Penerbit kepada \nPemegang sebagaimana dimaksud pada ayat (1) berupa: \na. biaya penggantian media Uang Elektronik untuk penggunaan \npertama kali atau penggantian media Uang Ele

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulati

sources [
  {
    "metadata": {
      "id": "doc:bi:5b09c79f187b4847b13295cfcc6d4add",
      "title": "PERATURAN BANK INDONESIA NOMOR: 12/10/PBI/2010 - PERUBAHAN KETIGA ATAS PERATURAN BANK INDONESIA NOMOR 5/13/PBI/2003 TENTANG POSISI DEVISA NETO BANK UMUM",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/d40ca498b87a457abcb60f9a356f9acdfaq_pbi_121011.pdf",
      "date": "1 Juli 2010",
      "type_of_regulation": "Peraturan Bank Indonesia",
      "sector": "Perbankan",
      "page_number": "2",
      "relevance_score": 0.04169577
    },
    "page_content": "teguran tertulis dan kewajiban membayar sebesar Rp250 juta per hari \npelanggaran dengan batas paling banyak sebesar Rp5 miliar per tahun \nkalender. Selain bentuk sanksi tersebut, terhadap  jenis pelanggaran tertentu \nakan dikenakan tambahan berupa dilakukan proses fit & proper test dan/atau \npenilaian tingkat kesehatan bank."
  },
  {
    "metadata": {
      "id": "doc:bi:2ef1e00dce924413abf97ee0cc20545e

...

sources [
  {
    "metadata": {
      "id": "doc:bi:07903c667796411495b9e96d1431f79a",
      "title": "PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 17 TAHUN 2023 TENTANG PENYELENGGARAAN BANK INDONESIA-FAST PAYMENT",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/PADG_172023.pdf",
      "date": "28 November 2023",
      "type_of_regulation": "Peraturan Anggota Dewan Gubernur",
      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",
      "page_number": "96",
      "relevance_score": 0.00012243443
    },
    "page_content": "kepada nasabah X dengan mengirimkan CTR baru sejumlah \nRp10.000.000,00 (sepuluh juta rupiah) yang dikirimkan \nkepada nasabah X dengan membayar jasa, bunga, imbalan, \natau kompensasi yang berlaku di Bank A ditambah 200 basis \npoint (2%) untuk 3 (tiga) hari kalender."
  },
  {
    "metadata": {
      "id": "doc:bi:19e943a792784c07840f680fa413d00c",
      "title": "SURAT EDARAN BANK INDONESIA NOMOR 18/23/DSTA TANGGAL 26 OKTOBER 2016 PER

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']


sources [
  {
    "metadata": {
      "id": "doc:bi:5b09c79f187b4847b13295cfcc6d4add",
      "title": "PERATURAN BANK INDONESIA NOMOR: 12/10/PBI/2010 - PERUBAHAN KETIGA ATAS PERATURAN BANK INDONESIA NOMOR 5/13/PBI/2003 TENTANG POSISI DEVISA NETO BANK UMUM",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/d40ca498b87a457abcb60f9a356f9acdfaq_pbi_121011.pdf",
      "date": "1 Juli 2010",
      "type_of_regulation": "Peraturan Bank Indonesia",
      "sector": "Perbankan",
      "page_number": "2",
      "relevance_score": 0.0012943681
    },
    "page_content": "teguran tertulis dan kewajiban membayar sebesar Rp250 juta per hari \npelanggaran dengan batas paling banyak sebesar Rp5 miliar per tahun \nkalender. Selain bentuk sanksi tersebut, terhadap  jenis pelanggaran tertentu \nakan dikenakan tambahan berupa dilakukan proses fit & proper test dan/atau \npenilaian tingkat kesehatan bank."
  },
  {
    "metadata": {
      "id": "doc:bi:d59a43e3fc004a329c6f8f85c31176

Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']
Metadata key page_number not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['file_id', 'title', 'file_name', 'file_link', 'date', 'type_of_regulation', 'sector', 'standardized_extracted_file_name', 'standardized_file_name', 'page_number']


sources [
  {
    "metadata": {
      "id": "doc:bi:5b09c79f187b4847b13295cfcc6d4add",
      "title": "PERATURAN BANK INDONESIA NOMOR: 12/10/PBI/2010 - PERUBAHAN KETIGA ATAS PERATURAN BANK INDONESIA NOMOR 5/13/PBI/2003 TENTANG POSISI DEVISA NETO BANK UMUM",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/d40ca498b87a457abcb60f9a356f9acdfaq_pbi_121011.pdf",
      "date": "1 Juli 2010",
      "type_of_regulation": "Peraturan Bank Indonesia",
      "sector": "Perbankan",
      "page_number": "2",
      "relevance_score": 0.00019563534
    },
    "page_content": "teguran tertulis dan kewajiban membayar sebesar Rp250 juta per hari \npelanggaran dengan batas paling banyak sebesar Rp5 miliar per tahun \nkalender. Selain bentuk sanksi tersebut, terhadap  jenis pelanggaran tertentu \nakan dikenakan tambahan berupa dilakukan proses fit & proper test dan/atau \npenilaian tingkat kesehatan bank."
  },
  {
    "metadata": {
      "id": "doc:bi:5f018d1a5e1d4793b4dd458516859

In [16]:
import json

with open("constant/bi/dict_eval.json", "w") as outfile: 
    json.dump(d, outfile)

In [17]:
with open('./constant/bi/dict_eval.json') as json_file:
    d = json.load(json_file)

In [18]:
dataset = Dataset.from_dict(d)

df = dataset.to_pandas()
df.head()

,question,answer,contexts,ground_truth
0,What is the cash value of SDBI with a nominal ...,[Your answer here] \n\nSource: No relevant inf...,[Lampiran SE No. 13/ 3 /DPM Tanggal 4 Febru...,"The cash value of SDBI is Rp 983,957,394.64"
1,When do the obligations of a Provider set fort...,The obligations of a Provider set forth in Art...,[12A sehingga berbunyi sebagai berikut: \nPasa...,1 July 2014.
2,When did the Bank Indonesia Regulation Number ...,Bank Indonesia Regulation Number 13/4/PBI/2011...,[teguran tertulis dan kewajiban membayar sebes...,"January 21, 2011."
3,Who promulgated the Bank Indonesia Regulation ...,Bank Indonesia promulgated the Bank Indonesia ...,[surat pemberitahuan sanksi administratif beru...,Patrialis Akbar. (Darmin Nasution stipulated it)
4,What is the interest rate paid by Bank Indones...,Bank Indonesia Circular Letter No. 9/3/DPM dat...,"[Simpan Pinjam sebesar Rp25.000,00 (dua puluh ...",5.5% per annum.


In [19]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness], llm=llm_model, embeddings=embed_model)

Evaluating: 100%|██████████| 420/420 [02:41<00:00,  2.60it/s]


In [20]:
import numpy as np

result_dict = df.to_dict(orient='index')

# Convert any ndarray objects to list
for key in result_dict:
    for subkey in result_dict[key]:
        if isinstance(result_dict[key][subkey], (np.ndarray, list)):
            result_dict[key][subkey] = result_dict[key][subkey].tolist() if isinstance(result_dict[key][subkey], np.ndarray) else result_dict[key][subkey]

# Save the dictionary to a JSON file
with open("./constant/bi/eval_result.json", "w") as file:
    json.dump(result_dict, file, indent=4)

In [21]:
for key,val in score.items():
    print(f"{key}: {val}")

faithfulness: 0.31626984126984126
answer_relevancy: 0.5969228858275406
context_precision: 0.2800462962742901
context_recall: 0.38333333333333336
answer_similarity: 0.7904195024834048
answer_correctness: 0.2737945486374032
harmfulness: 0.0
